In [11]:
# Load necessary libraries
import pandas as pd
import os

# Define the file path
pkl_path = 'data/cleaned_space_decay.pkl'

# Check if file exists
if os.path.exists(pkl_path):
    sdf = pd.read_pickle(pkl_path)
    print("✅ Cleaned DataFrame loaded successfully!")
    print("Number of objects:", len(sdf))
else:
    print("❌ Pickle file not found. Please run data_cleaner.ipynb first.")


✅ Cleaned DataFrame loaded successfully!
Number of objects: 14372


In [12]:
sample = sdf.iloc[0]
sample

CCSDS_OMM_VERS                                                         2
COMMENT                                GENERATED VIA SPACE-TRACK.ORG API
CREATION_DATE                                        2021-11-01T06:46:11
ORIGINATOR                                                       18 SPCS
OBJECT_NAME                                              ARIANE 42P+ DEB
OBJECT_ID                                                      1992-072J
CENTER_NAME                                                        EARTH
REF_FRAME                                                           TEME
TIME_SYSTEM                                                          UTC
MEAN_ELEMENT_THEORY                                                 SGP4
EPOCH                                         2021-10-31T22:46:50.340864
MEAN_MOTION                                                       2.9217
ECCENTRICITY                                                    0.652893
INCLINATION                                        

In [13]:
from skyfield.api import load, EarthSatellite

# Step 1: Load timescale
ts = load.timescale()

# Step 2: Extract TLE lines from sample
line1 = sample['TLE_LINE1']
line2 = sample['TLE_LINE2']
name = sample['OBJECT_NAME']

# Step 3: Create the satellite object
satellite = EarthSatellite(line1, line2, name, ts)


In [14]:
satellite

<EarthSatellite ARIANE 42P+ DEB catalog #26741 epoch 2021-10-31 22:46:50 UTC>

In [15]:
# Get current time or specific time range
t = ts.now()  # current UTC time
geocentric = satellite.at(t)  # Returns a Geocentric position
print(geocentric)

<Geocentric ICRS position and velocity at date t center=399 target=-126741>


In [16]:
# Position in ECI frame (Earth-Centered Inertial)
position = geocentric.position.km  # [x, y, z] in kilometers
print("ECI Position (km):", position)

# Convert to Lat-Long-Altitude
subpoint = geocentric.subpoint()
print("Latitude:", subpoint.latitude.degrees)
print("Longitude:", subpoint.longitude.degrees)
print("Altitude (km):", subpoint.elevation.km)


ECI Position (km): [  669.4456167  25720.7167754  -4019.61801687]
Latitude: -8.888011169806774
Longitude: 87.27849568791179
Altitude (km): 19663.892116256364


In [17]:
def get_position(sample_object): 
    line1 = sample_object['TLE_LINE1']
    line2 = sample_object['TLE_LINE2']
    name = sample_object['OBJECT_NAME']
    ts = load.timescale()
    sample_satellite = EarthSatellite(line1, line2, name, ts)
    time_now = ts.now()
    geocentric = sample_satellite.at(time_now)
    subpoint = geocentric.subpoint()
    satallite_position = {
        'ECI Position (km)': geocentric.position.km,
        'Latitude': subpoint.latitude.degrees,
        'Longitude': subpoint.longitude.degrees,
        'Altitude': subpoint.elevation.km
    }
    return satallite_position


for i in range(0,3):
    sample_object= sdf.iloc[i]
    print(f"Object No.{i}, Name: {sample_object['OBJECT_NAME']}, Position : {get_position(sample_object)} ", end ="\n\n")

Object No.0, Name: ARIANE 42P+ DEB, Position : {'ECI Position (km)': array([  669.2739968 , 25720.59445794, -4019.59456699]), 'Latitude': np.float64(-8.888004222232372), 'Longitude': np.float64(87.27860632950176), 'Altitude': np.float64(19663.763273966484)} 

Object No.1, Name: SL-8 DEB, Position : {'ECI Position (km)': array([ 3541.44093746,  2824.81076569, -5834.4454539 ]), 'Latitude': np.float64(-52.22179693121349), 'Longitude': np.float64(37.25728656628265), 'Altitude': np.float64(1021.8024497582651)} 

Object No.2, Name: GSAT 1, Position : {'ECI Position (km)': array([24932.47261409, 32768.60711612,  6733.50379038]), 'Latitude': np.float64(9.384213713858292), 'Longitude': np.float64(51.54330288239114), 'Altitude': np.float64(35344.72517018785)} 



In [19]:


# Check which columns are available
print("Columns in dataset:", sdf.columns)

# Inspect unique values in the OBJECT_TYPE column
print("\nUnique Object Types:")
print(sdf["OBJECT_TYPE"].value_counts())

# Filter to get only debris
debris_sdf = sdf[sdf["OBJECT_TYPE"].str.upper().str.contains("DEBRIS")]

print(f"\nTotal objects: {len(sdf)}")
print(f"Total debris objects: {len(debris_sdf)}")
print(f"Percentage of debris: {len(debris_sdf) / len(sdf) * 100:.2f}%")


Columns in dataset: Index(['CCSDS_OMM_VERS', 'COMMENT', 'CREATION_DATE', 'ORIGINATOR',
       'OBJECT_NAME', 'OBJECT_ID', 'CENTER_NAME', 'REF_FRAME', 'TIME_SYSTEM',
       'MEAN_ELEMENT_THEORY', 'EPOCH', 'MEAN_MOTION', 'ECCENTRICITY',
       'INCLINATION', 'RA_OF_ASC_NODE', 'ARG_OF_PERICENTER', 'MEAN_ANOMALY',
       'EPHEMERIS_TYPE', 'CLASSIFICATION_TYPE', 'NORAD_CAT_ID',
       'ELEMENT_SET_NO', 'REV_AT_EPOCH', 'BSTAR', 'MEAN_MOTION_DOT',
       'MEAN_MOTION_DDOT', 'SEMIMAJOR_AXIS', 'PERIOD', 'APOAPSIS', 'PERIAPSIS',
       'OBJECT_TYPE', 'RCS_SIZE', 'COUNTRY_CODE', 'LAUNCH_DATE', 'SITE',
       'DECAY_DATE', 'FILE', 'GP_ID', 'TLE_LINE0', 'TLE_LINE1', 'TLE_LINE2'],
      dtype='object')

Unique Object Types:
OBJECT_TYPE
DEBRIS         8431
PAYLOAD        4950
ROCKET BODY     744
TBA             247
Name: count, dtype: int64

Total objects: 14372
Total debris objects: 8431
Percentage of debris: 58.66%
